# Refinery Model

## Getting Started

In [20]:
f = open('results.txt');
print(f.read())
f.close()


Hello, World




In [17]:
%%script glpsol --out output -m /dev/stdin -o out.txt -y results.txt

printf: "\nHello, World\n\n";
end;


<img src = "https://raw.githubusercontent.com/jckantor/CBE40455/master/img/refining_opt.jpg"  style="width:1000px"/>

## Simple Model

In [18]:
%%script glpsol -m /dev/stdin -o /dev/stdout

set PRODUCTS;
set CRUDES;
set INTERMEDIATES;
set UNITS;

set STREAMS := CRUDES union INTERMEDIATES union PRODUCTS;
    
param Required{PRODUCTS} >= 0;
param Available{CRUDES} >= 0;
param UnitCapacity{UNITS} >= 0;

var f{STREAMS} >= 0;
var u{UNITS} >= 0;

# Mass Balances

# Mixer/Splitter junctions
s.t. PC01 : f['Butane'] = f['Butane-D'] + f['Butane-R'] - f['Butane-G'];
s.t. PC02 : f['CrNaphtha'] = f['CrNaphtha-G'] + f['CrNaphtha-HO'] + f['CrNaphtha-D'];
s.t. PC03 : f['CrGO'] = f['CrGO-HO'] + f['CrGO-D']; 
s.t. PC04 : f['SGO'] = f['SGO-HO'] + f['SGO-D'];
    
# Distillation Yields
s.t. D01 : f['Butane-D'] = 0.03*f['C1'] + 0.05*f['C2'];
s.t. D02 : f['Naphtha'] = 0.15*f['C1'] + 0.20*f['C2'];
s.t. D03 : f['Residue'] = 0.40*f['C1'] + 0.35*f['C2'];
s.t. D04 : f['Gasoil'] = 0.15*f['C1'] + 0.10*f['C2'];
    
# Reformer
s.t. R01 : f['Butane-R'] = 0.15*f['Naphtha'];
s.t. R02 : f['Reformate'] = 0.85*f['Naphtha'];

# Catalytic Cracking
s.t. C01 : f['CrNaphtha'] = 0.40*f['Residue'];
s.t. C02 : f['CrGO'] = 0.35*f['Residue'];

# Desulfurization
s.t. S01 : f['SGO'] = f['Gasoil'];
    
# Blending
s.t. GB : f['Gasoline'] = f['Butane-G'] + f['Reformate'] + f['CrNaphtha-G'];
s.t. HB : f['HeatingOil'] = f['CrNaphtha-HO'] + f['CrGO-HO'] + f['SGO-HO'];
s.t. DB : f['Diesel'] = f['CrNaphtha-D'] + f['CrGO-D'] + f['SGO-D'];
    
# Product Specifications

s.t. Octane : 120*f['Butane-G'] + 100*f['Reformate'] + 74*f['CrNaphtha-G'] >= 94*f['Gasoline'];
s.t. Pvap : 60*f['Butane-G'] + 2.6*f['Reformate'] + 4.1*f['CrNaphtha-G'] <= 12.7*f['Gasoline'];
s.t. Vol : 105*f['Butane-G'] + 3*f['Reformate'] + 12*f['CrNaphtha-G'] >= 17*f['Gasoline'];
s.t. Sulfur : 0.12*f['CrNaphtha-D'] + 0.76*f['CrGO-D'] + 0.03*f['SGO-D'] <= 0.05*f['Diesel'];
    
# Product production constraints
s.t. Requirement {p in PRODUCTS} : f[p] >= Required[p];
s.t. Availability {c in CRUDES} : f[c] <= Available[c];
    
s.t. Distillation : u['ADU'] = sum{c in CRUDES} f[c];
s.t. Reforming : u['CRU'] = f['Naphtha'];
s.t. CatCracking : u['CCU'] = f['Residue'];
s.t. Desulfurization : u['HDS'] = f['Gasoil'];
s.t. GasolineBlending : u['GBL'] = f['Gasoline'];
s.t. HOBlending : u['HBL'] = f['HeatingOil'];
s.t. DieselBlending : u['DBL'] = f['Diesel'];
    
minimize cost: 2.10*u['ADU'] + 4.18*u['CRU'] + 2.04*u['HDS'] + 0.60*u['CCU'];
    
data;

param : CRUDES :  Available :=
    'C1'  250000
    'C2'  400000;
    
param : PRODUCTS : Required :=
    Butane         20000
    Gasoline       40000
    HeatingOil     42000
    Diesel         30000;

set INTERMEDIATES := 
    Butane-D
    Butane-R
    Butane-G
    
    Naphtha 
    Residue 
    Gasoil 
    Reformate
    
    CrNaphtha
    CrNaphtha-G
    CrNaphtha-HO
    CrNaphtha-D
    
    CrGO          # Cracked Gasoil
    CrGO-HO       # Cracked Gasoil to Heating Oil Blending
    CrGO-D        # Cracked Gasoil to Diesel Blending
    
    SGO           # Sweetened Gasoil
    SGO-HO        # Sweetened Gasoil to Heating Oil Blending
    SGO-D;        # Sweetened Gasoil to Diesel Blending
       
param : UNITS : UnitCapacity :=
    ADU              500000    # Atmospheric Distillation Unit
    CRU               30000    # Catalytic Reformer Unit
    HDS               40000    # Hydrodesulfurization Unit
    CCU               50000    # Catalytic Cracking Unit
    GBL              500000    # Gasoline Blending Unit
    HBL              500000    # Heating Oil Blending Unit
    DBL              500000;   # Diesel Blending Unit
    
end;


GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 -m /dev/stdin -o /dev/stdout
Reading model section from /dev/stdin...
Reading data section from /dev/stdin...
/dev/stdin:112: warning: final NL missing before end of file
112 lines were read
Generating PC01...
Generating PC02...
Generating PC03...
Generating PC04...
Generating D01...
Generating D02...
Generating D03...
Generating D04...
Generating R01...
Generating R02...
Generating C01...
Generating C02...
Generating S01...
Generating GB...
Generating HB...
Generating DB...
Generating Octane...
Generating Pvap...
Generating Vol...
Generating Sulfur...
Generating Requirement...
Generating Availability...
Generating Distillation...
Generating Reforming...
Generating CatCracking...
Generating Desulfurization...
Generating GasolineBlending...
Generating HOBlending...
Generating DieselBlending...
Generating cost...
Model has been successfully generated
GLPK Simplex Optimizer, v4.52
34 rows, 30 columns, 89 non-ze